In [1]:
 from pathlib import Path
from pprint import pprint

import numpy as np
import pandas as pd

from openoa.types.plant_v2 import PlantDataV3, PlantMetaData

In [2]:
fpath = Path("/Users/rhammond/Documents/GitHub/OpenOA/examples/data/la_haute_borne")
fn_scada = "la-haute-borne-data-2014-2015.csv"
fn_meter = "plant_data.csv"
fn_curtail = "plant_data.csv"
fn_reanalysis = "merra2_la_haute_borne.csv"
fn_asset = "la-haute-borne_asset_table.csv"

scada = pd.read_csv(fpath / fn_scada)
meter = pd.read_csv(fpath / fn_meter)
curtail = pd.read_csv(fpath / fn_curtail)
reanalysis = pd.read_csv(fpath / fn_reanalysis)
asset = pd.read_csv(fpath / fn_asset)

## TODO
 - read data from spark, csv, pandas
 - read from json, yaml, dict, and pre-loaded object
 - automatically calculate wind direction from u/v windspeed
 - call planetos api if API key is provided
 - support flags for if csv/planetos/data object/etc
 - will need plant coordinates
 - expand metadata to contain plant-level identifiers (latitude, longitude)
 - check against the -25 namings and (likely) adopt that naming convention for the plant data
   - map the input column names, and provide a method to provide them back as the original inputs
 - get the 0 notebook working, or at least as a means to understand what will be required for refactoring

In [3]:
column_mapping = dict(  # rename to metadata
    scada=dict(
        time="Date_time",
        id="Wind_turbine_name",
        power="P_avg",
        windspeed="Ws_avg",
#         wtur_wspd="Ws_avg",  # TODO: adopt the -25 naming
        wind_direction="Wa_avg",
#         status="?",
        pitch="Ba_avg",
        temperature="Ot_avg",
        frequency="10T",
    ),
    meter=dict(
        time="time_utc",
        energy="net_energy_kwh",
    ),
    curtail=dict(
        time="time_utc",
        curtailment="curtailment_kwh",
        availability="availability_kwh",
        net_energy="net_energy_kwh",
        frequency="10T",
    ),
    reanalysis=dict(
        time="datetime",
        windspeed="windspeed_ms",
        windspeed_u="u_ms",
        windspeed_v="v_ms",
        temperature="temperature_K",
        rho="rho_kgm",
        frequency="10T",
#         product="merra2",
    ),
    asset=dict(
        id="id",
        latitude="latitude",
        longitude="longitude",
        rated_power="rated_power_kw",
#         type="?",
    ),
)

In [4]:
plant_meta = PlantMetaData.from_dict(column_mapping)

In [5]:
plant_meta.scada.dtypes

{'time': numpy.datetime64,
 'id': str,
 'power': float,
 'windspeed': float,
 'wind_direction': float,
 'status': str,
 'pitch': float,
 'temperature': float}

In [6]:
plant = PlantDataV3(
    metadata=column_mapping,
    scada=scada,
    meter=meter,
    curtail=curtail,
    reanalysis={"merra2": reanalysis, "era5": reanalysis},  # preferred, and enable API pulling
    asset=asset,
    analysis_type="MonteCarloAEP"
)

ValueError: Input data could not be converted to pandas

In [ ]:
plant.analysis_type = "all"
plant.validate()

In [ ]:
plant.analysis_type = "TurbineLongTermGrossEnergy"
plant.validate()

In [ ]:
plant.analysis_type = "ElectricalLosses"
plant.validate()

In [ ]:
ANALYSIS_REQUIREMENTS = {
    "MonteCarloAEP": {
        "meter": {
            "columns": ["energy"],
            "freq": ("MS", "D", "H", "T"),
        },
        "curtail": {
            "columns": ["availability", "curtailment"],
            "freq": ("MS", "D", "H", "T"),
        },
        "reanalysis": {
            "columns": ["windspeed", "rho"],
            "conditional_columns": {
                "reg_temperature": ["temperature"],
                "reg_winddirection": ["windspeed_u", "windspeed_v"],
            },
        },
    },
    "TurbineLongTermGrossEnergy": {
        "scada": {
            "columns": ["id", "windspeed", "power"],  # TODO: wtur_W_avg vs energy_kwh ?
            "freq": ("D", "H", "T"),
        },
        "reanalysis": {
            "columns": ["windspeed", "wind_direction", "rho"],
        },
    },
    "ElectricalLosses": {
        "scada": {
            "columns": ["energy"],
            "freq": ("D", "H", "T"),
        },
        "meter": {
            "columns": ["energy"],
            "freq": ("MS", "D", "H", "T"),
        },
    },
}